In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import sklearn
import timescaledb_model as tsdb
from analyze import get_files_infos_df, store_file

Logs of timescaledb_model go to /tmp/bourse.log


In [ ]:
# files_infos_df = get_files_infos_df()
# files_infos_df.to_pickle('files_infos_df.pkl')

## Files Dataframe - Filtering
1 hour for each Last day of Each month

In [3]:
files_infos_df = get_files_infos_df("./files_infos.pkl")

In [ ]:
files_infos_df["year_month"] = files_infos_df.index.to_period("M")
files_infos_df["date"] = files_infos_df.index.date
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"], files_infos_df["prefix"]]).last()["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["prefix"]]).first().reset_index()

## PEA PME - Symbol NF handling

In [5]:
df =store_file(files_infos_df[files_infos_df["prefix"] == "peapme"]["path"].iloc[0])

In [6]:
df["prefix"] = df["symbol"].apply(lambda x: x[0:3])

In [7]:
map_prefix_to_symbol = {
    "1rP": lambda x: x[3:8],
    "1rA": lambda x: x[3:8],
    "1rE": lambda x: x[4:9],
    "FF1": lambda x: x.split("_")[1],
}

df["symbol_nf"] = df.apply(
    lambda x: map_prefix_to_symbol.get(x["prefix"], lambda x: np.nan)(x["symbol"]),
    axis=1,
)